<h1>Extrakcia menných entít z odpovedí Geodotazníku</h1>

<h3>Import knižníc</h3>

In [13]:
import string
import pandas as pd
from collections import Counter
from nltk import ngrams
from os import path
import re

<h3>Nastavenie potrebných premenných pre ukladanie</h3>

In [14]:
working_dir = r"D:\\PRACOVNÁ PLOCHA - premiestnene z C na D\\skola\\lia\\projekt3"
file_name = r"otazka2.xlsx"
output_file_name = "ner_answers.tsv"
file_path = path.join(working_dir, file_name)   # cesta k excel súboru s odpoveďami na otázku 2
output_file_path = path.join(working_dir, output_file_name) # cesta k súboru, kam budeme výsledky ukladať 

In [15]:
data = pd.read_excel(file_path,header=0) # načítanie dát

<h3>Vyčistenie dát</h3>

In [16]:
def clear_data(row):
    """
    funkcia zmaže prebytočné znaky v odpovediach
    """
    row_str = row[0]
    row_str = row_str.replace("\n", " ")
    row_str = row_str.replace("\t", " ")
    row_str = row_str.replace("\xa0", " ")
    return row_str

In [17]:
data = data.apply(clear_data, axis=1)
data.head() # ukážka dát

0    Turistické mapy   Mapu sídel  Mapy hustoty zal...
1                                    Interaktivní mapy
2                                      Mapové aplikace
3                                mapy.cz  Google Earth
4                          Seznam - mapy  Google Earth
dtype: object

<h3>Extrakcia možných menných entít</h3>
<p><em>Možná menná entita</em> -- čokoľvek, čo začína veľkým písmenom</p>

In [18]:
def lower_case_words_in_list(lst):
    """
    funkcia zmení všetky slová v zozname na malé písmená
    """
    return [word.lower() for word in lst]
    
def extract_possible_named_entities_ngrams(data_str, n=1,lw=False):
    """
    funkcia nájde všetky n-gramy, ktoré obsahujú možné menné entity (slová s veľkým písmenom na začiatku)
    """
    possible_named_entities_ngrams = []
    for gram in ngrams(data_str.split(),n):
        ngram_string = " ".join(gram)
        word_with_upper_case_beginning =  r"[A-Z].+\s*"# r"[A-Z].*?\s"
        possible_named_entities_ngram = re.findall(string=ngram_string, pattern=word_with_upper_case_beginning)
        if possible_named_entities_ngram:
            possible_named_entities_ngrams.append(ngram_string)
    if lw:
        possible_named_entities_ngrams = lower_case_words_in_list(possible_named_entities_ngrams)
    return possible_named_entities_ngrams


In [19]:
data_str = " ".join(data.to_list())
data_str_without_punctuation = data_str.translate(str.maketrans("","", string.punctuation))
poss_named_entities = pd.DataFrame(Counter(extract_possible_named_entities_ngrams(data_str_without_punctuation, 1, lw=True)).items())
poss_named_entities_bigrams = pd.DataFrame(Counter(extract_possible_named_entities_ngrams(data_str_without_punctuation, 2, lw=True)).items())
poss_named_entities_trigrams =  pd.DataFrame(Counter(extract_possible_named_entities_ngrams(data_str_without_punctuation, 3, lw=True)).items())
poss_named_entities_quadgrams = pd.DataFrame(Counter(extract_possible_named_entities_ngrams(data_str_without_punctuation, 4, lw=True)).items())


<h3>Extrakcia webových stŕanok</h3>
<p><em>Webová stŕanka</em> -- čokoľvek, čo v sebe zahŕňa symbol <strong>.</strong> (bodka)</p>

In [20]:
def find_web_pages(data_str):
    """Nájde všetky výskyty slov, ktoré obsahujú bodku"""
    return re.findall(string=data_str, pattern=r"\w+?\.\w+")

In [21]:
web_pages = pd.DataFrame(Counter(lower_case_words_in_list(find_web_pages(data_str))).items()).sort_values(by=1, ascending=False)

In [22]:
web_pages.head() # ukážka dát

,0,1
0,mapy.cz,133
1,www.mapy,8
13,maps.google,6
2,maps.com,5
7,google.cz,5


<h3>Uloženie dát do tsv súboru</h3>
<p>Dáta ukladáme postupne zámenou argumentu funkcie save_data_to_tsv.</p>

In [23]:
save_data_to_tsv = lambda pandas_dataframe: pandas_dataframe.to_csv(output_file_path,sep="\t", index=False, header=False)

In [24]:
save_data_to_tsv(web_pages) # uložené dáta si prekopírovaním zobrazíme v Exceli